`rich.theme.Theme` Notes
==============================================================================

Experimenting with [`rich.theme.Theme`][]. In specific, how higher-order
resolution works (`A → B → C`), and setting/overriding ANSI named colors,
particularly with regards to logging in Jupyter notebooks (such as this).

[`rich.theme.Theme`]: https://rich.readthedocs.io/en/latest/reference/theme.html#rich.theme.Theme

First let's get some imports out of the way...

In [1]:
from rich.console import Console
from rich.theme import Theme
from rich.default_styles import DEFAULT_STYLES

Rich `Console` and `Theme`
------------------------------------------------------------------------------

`c_0` is our most default of consoles. It uses `DEFAULT_STYLES` as the sole
`Theme` in the theme stack, which maps _some_ ANSI color names to their ANSI
color values, like

```python
DEFAULT_STYLES: Dict[str, Style] = {
    # ...
    "black": Style(color="black"),
    "red": Style(color="red"),
    "green": Style(color="green"),
    "yellow": Style(color="yellow"),
    "magenta": Style(color="magenta"),
    "cyan": Style(color="cyan"),
    "white": Style(color="white"),
    # ...
}
```

Why it maps some and not others I couldn't tell ya, but the important thing to
recognize is that overriding (or adding missing) entries will not change the
ANSI colors, because something like `red: Style(color="red")` is just mapping
the name `red` to a _style_ that is simply the _color_ `red`.

How the _color_ `red` is displayed is a property of the terminal; `rich` sends
the ANSI code for `red` to the terminal and how that is displayed is out of `rich`'s hands.

Let's get a baseline for displaying colored text:

In [2]:
c_0 = Console()
assert c_0._theme_stack._entries == [DEFAULT_STYLES]
c_0.print("[red]red[/] [green]green[/] [blue]blue[/]")

red green blue

The _style_/_color_ difference is easy to see in an example where we override
the `red` _style_, and also create a new _style_ `also_red` that uses the
_color_ `red` — text marked up with `[red]...[/]` will use the `red:` override
(pure green in this case), but text marked up with `[also_red]...[/]` will
render as the same dull red from the `c_0` example.

In [3]:
c_1 = Console(
    theme=Theme(
        {
            "red": "#00FF00",
            "also_red": "red",
        }
    )
)
c_1.print("[red]red[/] [also_red]also_red[/]")

red also_red

This makes it clear that there is no multi-order resolution; we can not use a
_style_ name on the right-hand/value side of a theme mapping, we can only use
explicit colors there. The confusion stemmed from things like `red` being a
named _style_ and a _color_.

[Console][] has a `force_jupyter` option, which:

> Enable/disable Jupyter rendering, or None to auto-detect Jupyter. Defaults to
> None.

But forcing said Jupyter rendering does not seem to do anything for ANSI color
output; the blue is still largely unreadable against a dark notebook background.

[Console]: https://rich.readthedocs.io/en/latest/reference/console.html#rich.console.Console

In [4]:
c_jy = Console(force_jupyter=True)
c_jy.print("[red]red[/] [green]green[/] [blue]blue[/]")

red green blue

IDE Terminal Colors 
------------------------------------------------------------------------------

So what can be done here? How the ANSI colors are rendered has to be a property
of Jupyter or VSCode/Cursor.

I tried adjusting the IDE terminal colors with settings like

```json
"workbench.colorCustomizations": {
    "terminal.ansiBlue": "#7aa2ff",
    "terminal.ansiBrightBlue": "#9bbcff",
}
```

But that has no effect on "terminal" output in the notebooks.

Jupyter `<style>`
------------------------------------------------------------------------------

The A1 suggested overriding the `ansi-` CSS styles, but the VSCode/Cursor
Jupyter extension doesn't seem to _use_ those when rendering ANSI color-coded
output.

In [5]:
from IPython.core.display import HTML

HTML("""
<style>
    /* ANSI color overrides for Jupyter output */
    .ansi-red-fg { color: #E75C58; }
    .ansi-green-fg { color: #00A250; }
    .ansi-blue-fg { color: #208FFB; }

    /* Bright/bold variants */
    .ansi-bright-red-fg { color: #B22B31; }
    .ansi-bright-green-fg { color: #007427; }
    .ansi-bright-blue-fg { color: #0065CA; }

    /* Background colors */
    .ansi-red-bg { background-color: #E75C58; }
    .ansi-green-bg { background-color: #00A250; }
    .ansi-blue-bg { background-color: #208FFB; }
</style>
""")

c_0.print("[red]red[/] [green]green[/] [blue]blue[/]")

red green blue

Inspected Rendering
------------------------------------------------------------------------------

Inspecting the above "red blue green" element, we can see that it is not
applying `ansi-` CSS classes, it is specifying the color directly in the `style`
properties (whitespace-adjusted for easy reading):

```html
<div class="output_html" tabindex="0">
  <pre
    style="
      white-space: pre;
      overflow-x: auto;
      line-height: normal;
      font-family: Menlo, 'DejaVu Sans Mono', consolas, 'Courier New',
        monospace;
    "
  >
    <span style="color: #800000; text-decoration-color: #800000">
      red
    </span> 
    <span style="color: #008000; text-decoration-color: #008000">
      green
    </span> 
    <span style="color: #000080; text-decoration-color: #000080">
      blue
    </span>
  </pre>
</div>
```

There's that damn-near-invisible `#000080` blue.

Work Around
------------------------------------------------------------------------------

At this I'm content to say that adjusting the Jupyter ASCI colors in not
practical to chase down. What do we need to do it mitigate bad ANSI colors?     

1.  Only use theme styles, like

    ```python
    style = "log.path"
    style = "log.name"
    ```

I think that's it. This ensures that a user can override whatever key point to
`blue` or whatever is problematic with a better hex color value. We already
extend the `log.` theme namespace with `log.name`, `log.class`, `log.data.name`,
etc., we can add more of those as needed.

I added a `splatlog.rich.override_ansi_colors` function to automate the override procedure, replacing named ANSI colors on `splatlog.rich.THEME`:

In [6]:
from rich.markdown import Markdown
from rich.console import Console
import splatlog

splatlog.setup(
    level="info",
    console=Console(
        theme=splatlog.rich.override_ansi_colors(
            blue="#509dea", bright_blue="#439af4"
        )
    ),
)
LOG = splatlog.getLogger(__name__)

LOG.info(
    "Test test",
    some="thing",
    markdown=Markdown("Link: <http://example.com>"),
)

INFO      __main__                                                                                                 
msg       Test test                                                                                                
data      some        str          thing                                                                           
          markdown                 Link: ]8;id=660854;http://example.com\http://example.com]8;;\